In [ ]:
import pandas as pd
import snappy
import os
from itertools import groupby
import pickle
import numpy as np

In [ ]:
#option d'affichage pour les df
pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [ ]:
#choix des colonnes pour l'étude
col = ['time', 'unknown_IMISSIONTRAINNUMBER', 'WC_CAR01_LCST_ISYSTEMLIFESIGN', 'WC_CAR03_LCST_ISYSTEMLIFESIGN', 'WC_CAR05_LCST_ISYSTEMLIFESIGN', 'WC_CAR07_LCST_ISYSTEMLIFESIGN', 'WC_CAR01_LCST_IWSUTANKLEVEL',
'WC_CAR03_LCST_IWSUTANKLEVEL', 'WC_CAR05_LCST_IWSUTANKLEVEL', 'WC_CAR07_LCST_IWSUTANKLEVEL', 'WC_CAR01_LCST_IFWTANKCONTENT', 'WC_CAR03_LCST_IFWTANKCONTENT', 'WC_CAR05_LCST_IFWTANKCONTENT',
'WC_CAR07_LCST_IFWTANKCONTENT', 'WC_CAR01_LCST_IWWTANKCONTENT', 'WC_CAR03_LCST_IWWTANKCONTENT', 'WC_CAR05_LCST_IWWTANKCONTENT', 'WC_CAR07_LCST_IWWTANKCONTENT', 'WC_CAR01_LCST_IWCWORKTIME',
'WC_CAR03_LCST_IWCWORKTIME', 'WC_CAR05_LCST_IWCWORKTIME', 'WC_CAR07_LCST_IWCWORKTIME', 'WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE',
'WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR01_LCST_IDOORCYCLCNT', 'WC_CAR03_LCST_IDOORCYCLCNT', 'WC_CAR05_LCST_IDOORCYCLCNT',
'WC_CAR07_LCST_IDOORCYCLCNT', 'WC_CAR01_LCST_IFLUSHCNT', 'WC_CAR03_LCST_IFLUSHCNT', 'WC_CAR05_LCST_IFLUSHCNT', 'WC_CAR07_LCST_IFLUSHCNT', 'WC_CAR01_LCST_IEJECTORCNT', 
'WC_CAR03_LCST_IEJECTORCNT', 'WC_CAR05_LCST_IEJECTORCNT', 'WC_CAR07_LCST_IEJECTORCNT',  'WC_CAR01_LCST_IWATERTAPCNT', 'WC_CAR03_LCST_IWATERTAPCNT', 'WC_CAR05_LCST_IWATERTAPCNT', 
'WC_CAR07_LCST_IWATERTAPCNT', 'WC_CAR01_LCST_IDELIMINGCNT', 'WC_CAR03_LCST_IDELIMINGCNT', 'WC_CAR05_LCST_IDELIMINGCNT', 'WC_CAR07_LCST_IDELIMINGCNT', 'WC_CAR01_LCST_IDEFROSTCNT', 
'WC_CAR03_LCST_IDEFROSTCNT', 'WC_CAR05_LCST_IDEFROSTCNT', 'WC_CAR07_LCST_IDEFROSTCNT', 'WC_CAR01_LCST_IFWT05CNT', 'WC_CAR03_LCST_IFWT05CNT', 'WC_CAR05_LCST_IFWT05CNT', 'WC_CAR07_LCST_IFWT05CNT',
'WC_CAR01_LCST_IWWT95CNT', 'WC_CAR03_LCST_IWWT95CNT', 'WC_CAR05_LCST_IWWT95CNT', 'WC_CAR07_LCST_IWWT95CNT', 'WC_CAR01_LCST_FWCOUTOFSERVICE', 'WC_CAR03_LCST_FWCOUTOFSERVICE', 
'WC_CAR05_LCST_FWCOUTOFSERVICE', 'WC_CAR07_LCST_FWCOUTOFSERVICE', 'WC_CAR01_LCST_IWCREADY', 'WC_CAR03_LCST_IWCREADY', 'WC_CAR05_LCST_IWCREADY', 'WC_CAR07_LCST_IWCREADY', 
'WC_CAR01_LCST_FFWTEMPTY', 'WC_CAR03_LCST_FFWTEMPTY', 'WC_CAR05_LCST_FFWTEMPTY', 'WC_CAR07_LCST_FFWTEMPTY']


In [ ]:
#adresse des données
REP_DATA = 'D:/ESTACA/4A/Projet industriel/Données/Données_2023_03_13'

In [ ]:
#liste des répertoires de données (contenant les fichiers parquet)
content_list = os.listdir(REP_DATA)

In [ ]:
#création d'un dictionnaire pour les noms de rames (clé = rame, valeur = liste des répertoires)
#on ne garde que la premiere partie du nom des fichiers
racine_dict = {}
for racine, group_rep in groupby(content_list, lambda nom: nom.split('_')[0]):
    racine_dict[racine] = list(group_rep)

In [ ]:
racine_dict

In [ ]:
def cnt_missions(df):
    df['nombre_mission'] = 0
    for t in range(0, len(df)-1):
        if df.unknown_IMISSIONTRAINNUMBER[t]!=df.unknown_IMISSIONTRAINNUMBER[t+1]:
            df['nombre_mission'][t] = 1

In [ ]:
def remplissage_WSU(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_remplissageWSU'] = 0
        for t in range(0, len(df)-1):
            if df['WC_'+voiture+'_LCST_IWSUTANKLEVEL'][t]<df['WC_'+voiture+'_LCST_IWSUTANKLEVEL'][t+1]:
                df['WC_'+voiture+'_LCST_remplissageWSU'][t] = 1

In [ ]:
def consommation(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_consommation'] = df['WC_'+voiture+'_LCST_IWATERTAPCNT']*0.4+1.14*df['WC_'+voiture+'_LCST_remplissageWSU']

In [ ]:
def remplissage_instantane(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_remplissageInstantané%'] = (df['WC_'+voiture+'_LCST_IFLUSHCNT']*0.8*100)/315

In [ ]:
#création d'un dictionnaire de df en fonction des rames (clé = rame, valeur = df de données)
#lecture de tous les fichiers qui se rapportent à la même rame
#suppression de la dernière ligne car que des NaN dedans
#conversion du code mission en int pour que les 16 chiffres s'affichent
dict_df = {}
for rac, reps in racine_dict.items():
    df_list = []
    for rep in reps:
        df_temp = pd.read_parquet(REP_DATA + '/' + rep + '/TT_IP.parquet')
        df_temp = df_temp.loc[:, col].iloc[:-1]
        df_temp.unknown_IMISSIONTRAINNUMBER = df_temp.unknown_IMISSIONTRAINNUMBER.astype(np.int64)
        df_temp = df_temp.rename(columns={"time":'x_time'})
        df_list.append(df_temp)
        #df_temp.drop(df_temp.loc[df_temp['unknown_IMISSIONTRAINNUMBER']==0].index, inplace=True)
    df_concat = pd.concat(df_list, ignore_index=True)
    cnt_missions(df_concat)
    remplissage_WSU(df_concat)
    consommation(df_concat)
    remplissage_instantane(df_concat)
    dict_df[rac] = df_concat

In [ ]:
#sauvegarde du dictionnaire de df des rames en pickle
with open('dict_df.pkl', 'wb') as file:      
    pickle.dump(dict_df, file)

In [ ]:
#création du dictionnaire de df en fonction des missions de chaque rame (clé : rame, valeur : dictionnaire(clé = mission, valeur = df de données))
#lecture de toutes les missions possible d'une rame
dict_missions = {}
for rame, datas in dict_df.items():
    dict_mission_rame = {}
    missions = datas.unknown_IMISSIONTRAINNUMBER.unique()
    for mission in missions :
        df_temp = datas.query('unknown_IMISSIONTRAINNUMBER == @mission') #@ car mission variable externe a df
        dict_mission_rame[mission] = df_temp
    dict_missions[rame]=dict_mission_rame

In [ ]:
#sauvegarde du dictionnaire des missions en fonction des rames dans un pickle
with open('dict_rame_missions.pkl', 'wb') as file:      
    pickle.dump(dict_missions, file)

In [ ]:
# exemple de résultat : affichage des missions 0 de la rame z5500503
dict_missions['z5500503'][3836323437392020]

In [ ]:
#exemple de résultat : affichage de toutes les données de z5500503
dict_df['z5500503']

In [ ]:
dict_df['z5500503'].describe()

In [ ]:
dict_df['z5500503'].plot(x='x_time', y='WC_CAR01_LCST_IDOORCYCLCNT')